In [1]:
!pwd

/home/giakhang/dev/multi_task_autopilot/YOLOF-Mask/test


In [1]:
import sys
import torch.nn as nn
import numpy as np
sys.path.append("/home/giakhang/dev/multi_task_autopilot/YOLOF-Mask")

In [2]:
import torch
from yolof_mask.modeling.backbone.convnext import ConvNeXt

In [11]:
convnext = ConvNeXt(out_features=["stage_1", "stage_2", "stage_3", "stage_4"])

In [12]:
convnext._out_feature_channels

{'stage_1': 96, 'stage_2': 192, 'stage_3': 384, 'stage_4': 768}

In [13]:
convnext._out_feature_strides

{'stage_1': 4, 'stage_2': 8, 'stage_3': 16, 'stage_4': 32}

In [14]:
convnext.output_shape()

{'stage_1': ShapeSpec(channels=96, height=None, width=None, stride=4),
 'stage_2': ShapeSpec(channels=192, height=None, width=None, stride=8),
 'stage_3': ShapeSpec(channels=384, height=None, width=None, stride=16),
 'stage_4': ShapeSpec(channels=768, height=None, width=None, stride=32)}

In [15]:
input = torch.randn(1, 3, 720, 1280)

In [16]:
output = convnext(input)

In [20]:
output['stage_4'].shape

torch.Size([1, 768, 22, 40])

In [3]:
convnext = ConvNeXt(
    in_chans=3, 
    depths=[3, 3, 27, 3], 
    dims=[128, 256, 512, 1024], 
    drop_path_rate=0., 
    layer_scale_init_value=1e-6, 
    out_features=["stage_4"],
    size_divisibility=32,
)

In [4]:
url = "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth"
checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth" to /home/giakhang/.cache/torch/hub/checkpoints/convnext_base_22k_224.pth
100%|██████████| 420M/420M [00:11<00:00, 39.4MB/s] 


In [5]:
state_dict = checkpoint["model"]
new_state_dict = {}

for key, value in state_dict.items():
    if "downsample_layers" in key:
        old_keys = key.split(".")
        new_keys = old_keys
        new_keys[1] = f"downsample_layer_{int(old_keys[1]) + 1}"
        new_key = ".".join(new_keys)
    elif "stages" in key:
        old_keys = key.split(".")
        new_keys = old_keys
        new_keys[1] = f"stage_{int(old_keys[1]) + 1}"
        new_key = ".".join(new_keys)
    else:
        continue
    new_state_dict[new_key] = value

In [6]:
convnext.load_state_dict(new_state_dict, strict=False)

_IncompatibleKeys(missing_keys=['norm_1.weight', 'norm_1.bias', 'norm_2.weight', 'norm_2.bias', 'norm_3.weight', 'norm_3.bias', 'norm_4.weight', 'norm_4.bias'], unexpected_keys=[])